In [14]:
import pandas as pd
from pathlib import Path
import re
import sqlite3

In [15]:
root = Path(r"P:\Projects_GW\State\Petroleum_Developments\_co_produced_water\peps_data")
db = sqlite3.connect(root / "peps.db")

In [22]:
date_paths = []
file_paths = {}
for path in root.glob("*"):
    m = re.match(r"\d\d\d\d-\d\d-\d\d", path.stem)
    if m:
        date_paths.append(path)
        file_paths[path] = []
for date_path in date_paths:
    for path in date_path.glob("*.xlsx"):
        file_paths[date_path].append(path)
        
latest_date_path = max(date_paths, key=lambda p: p.stem)
latest_file_paths = file_paths[latest_date_path]

In [33]:
metadata_df = pd.DataFrame({
    "path": [str(latest_date_path)],
    "path_date": [latest_date_path.stem],
}).to_sql("table_metadata", db, if_exists="replace")

date_cols = [
    'Month End', "Pick Date", "Logged Date", "Received Date",
    "Spudded Date", "Rig Release Date", "Open File Date",
    "Review Date", "Completion Date",
]

for i, path in enumerate(latest):
    print(f"{i}\t{path.name}")
    
for i, path in enumerate(latest):
    print("Processing", i, path.name)    
    df = pd.read_excel(path)
    for col in [c for c in df.columns if c in date_cols]:
        df[col] = df[col].dt.strftime("%Y-%m-%d")
    for col in [c for c in df.columns if "(ft)" in c or "(bbl)" in c or "(mmcf)" in c]:
        df = df.drop(col, axis=1)
    df.to_sql(path.stem, db, if_exists="replace")

0	CarolineC02MonthlyProductionData.xlsx
1	FormationTops.xlsx
2	LogDigital.xlsx
3	LogsPrintsAndImages.xlsx
4	MonthlyProductionByCompletion.xlsx
5	MonthlyProductionByField.xlsx
6	SAPetroleumWells.xlsx
7	WellCompletionReport.xlsx
Processing 0 CarolineC02MonthlyProductionData.xlsx
Processing 1 FormationTops.xlsx
Processing 2 LogDigital.xlsx
Processing 3 LogsPrintsAndImages.xlsx
Processing 4 MonthlyProductionByCompletion.xlsx
Processing 5 MonthlyProductionByField.xlsx
Processing 6 SAPetroleumWells.xlsx
Processing 7 WellCompletionReport.xlsx


In [83]:
df = pd.read_sql("""
select 
    "Prod-Basin",
    Field,
    "Co-Formation",
    Well,
    "Month End",
    strftime('%Y-%m', "Month End") as Month,
    "Water (m3)" as "Water (kL)",
    "Water (m3)" / "Stream Days" * 1000 / 86400 as "Water Rate (L/s)"
from MonthlyProductionByCompletion
group by "Prod-Basin", Field, "Co-Formation", Well
""", db)
df

,Prod-Basin,Field,Co-Formation,Well,Month End,Month,Water (kL),Water Rate (L/s)
0,Cooper,Acrasia,Tinchoo Fm,Acrasia 005,2011-01-31,2011-01,0.00,0.000000
1,Cooper,Acrasia,Tinchoo Fm,Acrasia 006,2014-04-30,2014-04,1542.90,0.626584
2,Cooper,Acrasia,Tinchoo Fm,Acrasia 007,2014-01-31,2014-01,0.25,0.000499
3,Cooper,Acrasia,Tinchoo Fm (Lower),Acrasia 002,2002-09-30,2002-09,0.00,0.000000
4,Cooper,Acrasia,Tinchoo Fm (Upper),Acrasia 001,2002-07-31,2002-07,0.39,0.000903
...,...,...,...,...,...,...,...,...
3106,Warburton,Koree South,Kalladeina,Koree South 002,2017-08-31,2017-08,4.40,0.012731
3107,Warburton,Moomba North,WARBURTON,Moomba 134,2001-08-31,2001-08,0.00,0.000000
3108,Warburton,Moomba North,WARBURTON,Moomba 197,2015-01-31,2015-01,0.00,0.000000
3109,Warburton,Waukatanna,Pando Fm,Waukatanna 005,2013-03-31,2013-03,0.00,0.000000
